Reading and analyzing New York Taxi data set 

In [1]:
import numpy as np
import pandas as pd

In [31]:
taxidata = pd.read_csv('C:/Users/Madhumita.Ganesan/Desktop/DataScience/data/nyctaxi/nycTaxiData/trip_fare_500k.csv',engine='python')


taxidata.pickup_datetime = pd.to_datetime(taxidata.pickup_datetime,format="%Y-%m-%d %H:%M:%S")
taxidata.dtypes

medallion                  object
hack_license               object
vendor_id                  object
pickup_datetime    datetime64[ns]
payment_type               object
fare_amount               float64
surcharge                 float64
mta_tax                   float64
tip_amount                float64
tolls_amount              float64
total_amount              float64
dtype: object

In [8]:
taxidata.head(2)
#type(taxidata)

,medallion,hack_license,vendor_id,pickup_datetime,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,CMT,2013-01-01 15:11:48,CSH,6.5,0.0,0.5,0.0,0.0,7.0
1,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,2013-01-06 00:18:35,CSH,6.0,0.5,0.5,0.0,0.0,7.0


In [13]:
##Most Expensive / Least Expensive trip taken

most_expensive_data = taxidata.total_amount.max()
least_expensive_data = taxidata.total_amount.min()

print("Most expensive trip is :", most_expensive_data, "\n Least expensive trip is : ", least_expensive_data)
type(most_expensive_data)

Most expensive trip is : 460.5 
 Least expensive trip is :  2.5


numpy.float64

In [33]:
#Does the overall total_amount paid per ride correlate with tip_amount per ride?
totripcorr = taxidata.total_amount.corr(taxidata.tip_amount)
print(totripcorr)

0.6708753071402004


In [39]:
##Does it correlate when you remove all rides with unknown payment_type?
taxidata_filtered=taxidata[taxidata.payment_type!='UNK']

tripcorr = taxidata_filtered.total_amount.corr(taxidata_filtered.tip_amount)
print(tripcorr)

0.6703435782941249


##Calculate the average cost of a trip in this dataset given the following conditions:
1) Across the whole dataset
2) Across the whole dataset when the payment_type is known (not UNK)
3) For each payment_type
4) Which payment_type had the highest average cost?
5) Which payment_type had the largest spread in how much people paid (largest standard deviation)?
6) Which payment_type had the most generous people (had the highest average tip), including unknown payment types?
7) What hour in the day were people most generous, on average, when they got into a cab?
8) What hour of the day did people fluctuate the most in terms of tips? That is, do some hours lead to unpredictable tip amounts?

In [53]:
##Average cost across whole dataset
avgcost = taxidata.total_amount.mean()
print("Average cost accross whole dataset : " ,avgcost)

avgcost_notunk = taxidata_filtered.total_amount.mean()
print("Average cost for dataset when payment type is not UNK : ",avgcost_notunk)

##Average cost for each payment type
taxigroup =taxidata.groupby('payment_type')
avg_payment_cost = taxigroup.total_amount.mean()
print("Average payment cost by payment type is :")
print(avg_payment_cost)
##Payment type with highest average cost

highest_avg_cost_ptype = avg_payment_cost.sort_values(ascending=False).head(1)
print("Payment type with highest average cost : \n",highest_avg_cost_ptype)

##Payment type wit larget spread in cost
taxi_std = taxigroup.total_amount.std()
print("Payment type with largest spread in cost is : \n")
print(taxi_std.sort_values(ascending=False).head(1))

Average cost accross whole dataset :  14.170294440000001
Average cost for dataset when payment type is not UNK :  14.16302480467147
Average payment cost by payment type is :
payment_type
CRD    16.371267
CSH    11.616747
DIS     5.750000
NOC     3.000000
UNK    22.116674
Name: total_amount, dtype: float64
Payment type with highest average cost : 
 payment_type
UNK    22.116674
Name: total_amount, dtype: float64
Payment type with largest spread in cost is : 

payment_type
UNK    21.049277
Name: total_amount, dtype: float64


In [62]:
##Payment type with most generous tip
tip_avg = taxigroup.tip_amount.mean().sort_values(ascending=False)
print("Payment type with most generous tip: \n",tip_avg.head(1))

Payment type with most generous tip: 
 payment_type
UNK    3.430088
Name: tip_amount, dtype: float64


In [90]:
## What hour in the day were most people generous, on average,when they got into the cab

taxidata['hour'] = taxidata.pickup_datetime.dt.hour
hourgroup = taxidata.groupby('hour')
avgtip_amount = hourgroup.tip_amount.mean().sort_values(ascending=False)
#avg_tip_list=list(avgtip_amount)
print(avgtip_amount.head(1))

hour
5    1.942813
Name: tip_amount, dtype: float64


In [93]:
## What hour in the day did people fluctuate most on tips
hourgroup = taxidata.groupby('hour')
std_tipamt = hourgroup.tip_amount.std().sort_values(ascending=False)
print(std_tipamt)

hour
5     3.275576
4     2.659419
0     2.507986
16    2.446023
6     2.395520
17    2.370470
14    2.370127
15    2.363344
1     2.319315
23    2.286110
21    2.220345
19    2.122856
3     2.085183
13    2.082766
7     2.066681
22    2.037812
9     1.988527
12    1.954182
20    1.950661
10    1.913502
8     1.908732
11    1.894122
18    1.886830
2     1.869682
Name: tip_amount, dtype: float64


In [94]:
#Which person (hack_license) made the most money:
#1) In total
#2) On a per-trip basis, given that they took at least 20 trips
taxidata.columns

Index(['medallion', 'hack_license', 'vendor_id', 'pickup_datetime',
       'payment_type', 'fare_amount', 'surcharge', 'mta_tax', 'tip_amount',
       'tolls_amount', 'total_amount', 'hour'],
      dtype='object')

In [104]:
person_group = taxidata.groupby('hack_license')
max_money_person = person_group.total_amount.sum().sort_values(ascending=False)
#print(max_money_person.head(1))
print("Person who made max money in total \n",max_money_person[max_money_person == max_money_person.max()])

##On a per-trip basis given he made atleast 20 trips
mintripperperson = taxidata.groupby('hack_license').filter(lambda x:x.shape[0] >= 20)
pertrip_group=mintripperperson.groupby('hack_license')["total_amount"].mean()
print("Person atleast 20 trips max money is \n: ",pertrip_group[pertrip_group == pertrip_group.max()])


Person who made max money in total 
 hack_license
CFCD208495D565EF66E7DFF9F98764DA    2517.28
Name: total_amount, dtype: float64
Person atleast 20 trips max money is 
:  hack_license
5114DF85775775ED4F53235D8478E80B    53.466667
Name: total_amount, dtype: float64


In [118]:
##Does the number of trips a given cabbie takes (her/his experience) correlate with how well she/he is tipped? If so, in what direction?
groupsummary = taxidata.groupby("hack_license")["tip_amount"].agg([np.size,np.mean])
print(groupsummary.corr())
#groupsummary.head(4)

#Negative coefficient of correlation


          size      mean
size  1.000000 -0.291082
mean -0.291082  1.000000


In [119]:
#* Does the number of times a given cab is used correlate with how well the person driving the cab is tipped? That is, are there "lucky" cabs?

groupsummary = taxidata.groupby("medallion")["tip_amount"].agg([np.size,np.mean])
print(groupsummary.corr())
#groupsummary.head(4)

         size     mean
size  1.00000 -0.34822
mean -0.34822  1.00000


In [138]:
#* Which `vendor_id` had the highest average `surcharge` on a per-hour basis?
taxidata.columns
pervendoravg = taxidata.groupby(["vendor_id","hour"])["surcharge"].mean().unstack(level=1)
#print(pervendoravg)
pervendormean = pervendoravg.mean(axis=1)
print("Vendor with highest average surcharge is :\n",pervendormean[pervendormean == pervendormean.max()])

Vendor with highest average surcharge is :
 vendor_id
VTS    0.293235
dtype: float64


In [140]:
#  Which hour in the day: 
#  1. Did people most frequently take rides?

hourgroup=taxidata.groupby("hour").size()
maxhour=hourgroup[hourgroup==hourgroup.max()]
print("Hour of most frequent rides : ",maxhour)
#  2. Did people least frequently take rides?

hourgroup=taxidata.groupby("hour").size()
maxhour=hourgroup[hourgroup==hourgroup.min()]
print("Hour of least frequest rides : ",maxhour)

#  3. Had the largest number of unique cabs on the street?

uniquecabs=taxidata.drop_duplicates(["medallion","hour"])
uniqueperhour=uniquecabs.groupby("hour").size()
maxunique=uniqueperhour[uniqueperhour==uniqueperhour.max()]
print("Largest number of unique cabs per hour in the street : ",maxunique)
#  4. Had the least number of cabs in the street?
uniquecabs=taxidata.drop_duplicates(["medallion","hour"])
uniqueperhour=uniquecabs.groupby("hour").size()
maxunique=uniqueperhour[uniqueperhour==uniqueperhour.max()]
print("Largest number of unique cabs per hour in the street : ",maxunique)


#  5. What is the average number of cabs on the streets in NYC in each quarter of the day (at least in this dataset?)?
taxidata["quarter"]=pd.cut(taxidata.hour,[-1,5,11,17,np.inf])
uniquecabs=taxidata.drop_duplicates(["medallion","quarter"])
avgcabsquarter=uniquecabs.groupby("quarter").size()
print(avgcabsquarter)

quarter
(-1.0, 5.0]     5583
(5.0, 11.0]     6559
(11.0, 17.0]    7025
(17.0, inf]     6958
dtype: int64
